In [1]:
from policyengine_us import Microsimulation
from policyengine_core.reforms import Reform
import pandas as pd
import os

In [2]:
niit_reform = Reform.from_dict({
    "gov.contrib.biden.budget_2025.net_investment_income.rate": {
        "2024-01-01.2100-12-31": 0.012
    },
    "gov.contrib.biden.budget_2025.net_investment_income.threshold": {
        "2025-01-01.2025-12-31": 411000,
        "2026-01-01.2026-12-31": 418450,
        "2027-01-01.2027-12-31": 427500,
        "2028-01-01.2028-12-31": 435950,
        "2029-01-01.2029-12-31": 444275,
        "2030-01-01.2030-12-31": 453025,
        "2031-01-01.2031-12-31": 461975,
        "2032-01-01.2032-12-31": 471175,
        "2033-01-01.2033-12-31": 480625,
        "2034-01-01.2034-12-31": 490300,
        "2035-01-01.2100-12-31": 500200
    }
}, country_id="us")

# Medicare Reform
medicare_reform = Reform.from_dict({
    "gov.contrib.biden.budget_2025.medicare.rate": {
        "2024-01-01.2100-12-31": 0.012
    },
    "gov.contrib.biden.budget_2025.medicare.threshold": {
        "2025-01-01.2025-12-31": 411000,
        "2026-01-01.2026-12-31": 418450,
        "2027-01-01.2027-12-31": 427500,
        "2028-01-01.2028-12-31": 435950,
        "2029-01-01.2029-12-31": 444275,
        "2030-01-01.2030-12-31": 453025,
        "2031-01-01.2031-12-31": 461975,
        "2032-01-01.2032-12-31": 471175,
        "2033-01-01.2033-12-31": 480625,
        "2034-01-01.2034-12-31": 490300,
        "2035-01-01.2100-12-31": 500200
    }
}, country_id="us")

In [ ]:
def analyze_reforms():
    # Set up years to analyze
    years = range(2025, 2035)
    
    # Define reforms dictionary
    reforms = {
        'NIIT': niit_reform,
        'Medicare': medicare_reform
    }
    
    # Create or load results dataframe
    csv_path = 'tax_reform_impacts.csv'
    if os.path.exists(csv_path):
        results = pd.read_csv(csv_path, index_col=0)
        print("Loaded existing results from CSV")
    else:
        results = pd.DataFrame(index=list(years), columns=reforms.keys())
        print("Created new results DataFrame")
    
    # Initialize baseline simulation
    baseline = Microsimulation(dataset='enhanced_cps_2024')
    
    # Process each reform
    for reform_name, reform_policy in reforms.items():
        print(f"\nProcessing {reform_name} reform...")
        
        # Initialize reform simulation
        reform_sim = Microsimulation(reform=reform_policy, dataset='enhanced_cps_2024')
        
        # Calculate impacts for each year
        for year in years:
            # Skip if already calculated
            if year in results.index and pd.notna(results.at[year, reform_name]):
                print(f"Skipping {reform_name} year {year} - already calculated")
                continue
                
            print(f"Processing {reform_name} year {year}")
            try:
                baseline_income = baseline.calculate("household_net_income", period=year).sum()
                reformed_income = reform_sim.calculate("household_net_income", period=year).sum()
                
                # Calculate impact in billions
                impact = (reformed_income - baseline_income) / 1e9
                d.at[year, reform_name] = round(impact, 1)
                
                # Save after each calculation
                results.to_csv(csv_path)
                print(f"Saved results for {reform_name} year {year}")
                
            except Exception as e:
                print(f"Error processing {reform_name} year {year}: {str(e)}")
                # Save progress even if there's an error
                results.to_csv(csv_path)
                continue
    
    # Calculate 2025-34 total if we have all years for both reforms
    if all(pd.notna(results.loc[2025:2034].values.flatten())):
        results.loc['2025-34'] = results.loc[2025:2034].sum()
        results.to_csv(csv_path)
        print("Added 2025-34 total")
    
    return results


In [4]:
# Run analysis
results = analyze_reforms()
print("\nReform Impacts (Billions $):")
print(results)

Loaded existing results from CSV

Processing NIIT reform...
Skipping NIIT year 2025 - already calculated
Skipping NIIT year 2026 - already calculated
Skipping NIIT year 2027 - already calculated
Skipping NIIT year 2028 - already calculated
Skipping NIIT year 2029 - already calculated
Skipping NIIT year 2030 - already calculated
Skipping NIIT year 2031 - already calculated
Processing NIIT year 2032
Saved results for NIIT year 2032
Processing NIIT year 2033
Saved results for NIIT year 2033
Processing NIIT year 2034
Saved results for NIIT year 2034

Processing Medicare reform...
Processing Medicare year 2025
Saved results for Medicare year 2025
Processing Medicare year 2026
Saved results for Medicare year 2026
Processing Medicare year 2027


KeyboardInterrupt: 